# Coursera Capstone Project

## 1st Assg

In [26]:
import pandas as pd
import numpy as np
import json 


import requests 
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim 
from sklearn.cluster import KMeans
import folium 

In [25]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

Solving environment: \ 
  - anaconda::ca-certificates-2019.1.23-0, anaconda::openssl-1.1.1b-h7b6447c_1
  - anaconda::ca-certificates-2019.1.23-0, defaults::openssl-1.1.1b-h7b6447c_1
  - anaconda::openssl-1.1.1b-h7b6447c_1, defaults::ca-certificates-2019.1.23-0
  - defaults::ca-certificates-2019.1.23-0, defaults::openssl-1.1.1b-h7b6447cdone

## Package Plan ##

  environment location: /home/rahul/anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.3.9           |           py37_0         149 KB  conda-forge
    conda-4.7.12               |           py37_0         3.0 MB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                              

In [3]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## 2nd Assg

In [4]:
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

tables = pd.read_html(link,header=0)
table = tables[0]
table.tail()

,Postcode,Borough,Neighbourhood
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West
286,M8Z,Etobicoke,South of Bloor
287,M9Z,Not assigned,Not assigned


Dropping NA boroughs

In [5]:
table = table[table['Borough'] != 'Not assigned']
table.reset_index(drop=True,inplace=True)
table.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Combining cells with common postcode

In [19]:
foo = lambda a: ",".join(a) 
table=table.groupby(by=['Postcode','Borough']).agg({'Neighbourhood':foo})
table.head()

,,Neighbourhood
Postcode,Borough,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


Rearranging the columns

In [8]:
table.reset_index(level=0, inplace=True)
table.reset_index(level=0, inplace=True)
cols = ['Postcode','Borough','Neighbourhood']
table = table[cols]
table.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Replacing NA neighbourhoods

In [9]:
for index,row in table.iterrows():
    if(row['Neighbourhood'] == "Not assigned"):
        row['Neighbourhood'] = row['Borough'] 

In [10]:
table[table['Borough'] == "Queen's Park"]

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


In [11]:
table.shape

(103, 3)

#### Combining coorodinates csv

In [15]:
df = pd.read_csv("Geospatial_Coordinates.csv")

In [16]:
canada = pd.merge(table, df, left_on='Postcode', right_on='Postal Code')

In [ ]:
canada.rename(columns = {'Postcode':'Postalcode'})
del canada['Postal Code']

In [23]:
canada.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### Map Visualization

In [31]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [92]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(canada['Latitude'], canada['Longitude'], canada['Borough'], canada['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Function to structure API call

In [42]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            'SJI1FGMQ0NXZ14XECGFLEAHV5QU21SVQ0LVCVYEQUUJK5M4N', 
            'DVHMOLXVMANYG4C0UECBF4W4YWO5FQISFR1PDSYAJ14JGZHC', 
            '20180602', 
            lat, 
            lng, 
            radius, 
            25)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

API results

In [43]:


toronto_venues = getNearbyVenues(names=canada['Neighbourhood'],
                                   latitudes=canada['Latitude'],
                                   longitudes=canada['Longitude'])


Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [44]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [45]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 219 uniques categories.


Preprocessing

In [47]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
num_top_venues = 5
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
              venue  freq
0        Steakhouse  0.12
1             Hotel  0.08
2  Asian Restaurant  0.08
3       Coffee Shop  0.08
4        Food Court  0.04


----Agincourt----
                venue  freq
0              Lounge   0.2
1      Breakfast Spot   0.2
2        Skating Rink   0.2
3  Chinese Restaurant   0.2
4      Sandwich Place   0.2


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
         venue  freq
0  Coffee Shop  0.33
1         Park  0.33
2   Playground  0.33
3        Motel  0.00
4       Market  0.00


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                  venue  freq
0         Grocery Store   0.2
1   Fried Chicken Joint   0.1
2           Coffee Shop   0.1
3        Sandwich Place   0.1
4  Fast Food Restaurant   0.1


----Alderwood,Long Branch----
                venue  freq
0         Pizza Place  0.18
1                 Gym  0.09
2  Athletics & Sports  0.0

               venue  freq
0        Coffee Shop  0.33
1               Park  0.33
2  Convenience Store  0.33
3      Movie Theater  0.00
4     Massage Studio  0.00


----Emery,Humberlea----
                        venue  freq
0  Construction & Landscaping   0.5
1              Baseball Field   0.5
2                 Yoga Studio   0.0
3                       Motel   0.0
4                      Market   0.0


----Fairview,Henry Farm,Oriole----
            venue  freq
0  Clothing Store  0.16
1     Coffee Shop  0.08
2        Pharmacy  0.04
3        Tea Room  0.04
4   Smoothie Shop  0.04


----First Canadian Place,Underground city----
              venue  freq
0              Café  0.12
1       Coffee Shop  0.12
2        Restaurant  0.08
3        Steakhouse  0.08
4  Asian Restaurant  0.04


----Flemingdon Park,Don Mills South----
                 venue  freq
0                  Gym  0.09
1           Beer Store  0.09
2          Coffee Shop  0.09
3     Asian Restaurant  0.09
4  Sporting Goods Shop  

            venue  freq
0  Sandwich Place  0.13
1            Café  0.13
2     Coffee Shop  0.13
3     Pizza Place  0.09
4             Pub  0.04


----The Beaches----
               venue  freq
0  Health Food Store  0.25
1                Pub  0.25
2              Trail  0.25
3        Yoga Studio  0.00
4              Motel  0.00


----The Beaches West,India Bazaar----
               venue  freq
0     Sandwich Place  0.10
1        Pizza Place  0.10
2       Burger Joint  0.05
3     Ice Cream Shop  0.05
4  Food & Drink Shop  0.05


----The Danforth West,Riverdale----
                venue  freq
0    Greek Restaurant  0.28
1  Italian Restaurant  0.08
2      Ice Cream Shop  0.08
3         Yoga Studio  0.04
4           Bookstore  0.04


----The Junction North,Runnymede----
                  venue  freq
0         Grocery Store   0.2
1               Brewery   0.2
2     Convenience Store   0.2
3  Caribbean Restaurant   0.2
4           Pizza Place   0.2


----The Kingsway,Montgomery Road,Old Mill N

In [50]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [55]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Steakhouse,Asian Restaurant,Coffee Shop,Hotel,Greek Restaurant,Smoke Shop,Lounge,Speakeasy,Bar,Sushi Restaurant
1,Agincourt,Lounge,Skating Rink,Breakfast Spot,Chinese Restaurant,Sandwich Place,Women's Store,Dance Studio,Dog Run,Discount Store,Diner
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Coffee Shop,Playground,Park,Curling Ice,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Discount Store,Diner
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pharmacy,Fried Chicken Joint,Pizza Place,Sandwich Place,Liquor Store,Beer Store,Fast Food Restaurant,Coffee Shop,Frozen Yogurt Shop
4,"Alderwood,Long Branch",Pizza Place,Coffee Shop,Pub,Skating Rink,Pharmacy,Gym,Dance Studio,Sandwich Place,Athletics & Sports,Pool


#### Clustering

In [57]:
kclusters = 5

manhattan_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

kmeans.labels_[0:10] 

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [85]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = canada

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged = toronto_merged.dropna()

In [86]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [89]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,0.0,Coffee Shop,Playground,Park,Curling Ice,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Discount Store,Diner
23,North York,0.0,Park,Convenience Store,Bank,Women's Store,Dance Studio,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Discount Store
25,North York,0.0,Park,Food & Drink Shop,Women's Store,Curling Ice,Eastern European Restaurant,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant
30,North York,0.0,Airport,Park,Construction & Landscaping,Women's Store,Dance Studio,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Discount Store
31,North York,0.0,Grocery Store,Park,Shopping Mall,Bank,Hotel,Women's Store,Dance Studio,Eastern European Restaurant,Drugstore,Dog Run
40,East York,0.0,Park,Convenience Store,Coffee Shop,Women's Store,Dance Studio,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Discount Store
44,Central Toronto,0.0,Bus Line,Park,Dim Sum Restaurant,Swim School,Women's Store,Eastern European Restaurant,Drugstore,Dog Run,Discount Store,Diner
50,Downtown Toronto,0.0,Park,Playground,Trail,Building,Women's Store,Curling Ice,Drugstore,Dog Run,Discount Store,Diner
64,Central Toronto,0.0,Bus Line,Park,Jewelry Store,Trail,Sushi Restaurant,Women's Store,Eastern European Restaurant,Drugstore,Dog Run,Discount Store
72,North York,0.0,Pizza Place,Park,Pub,Japanese Restaurant,Women's Store,Curling Ice,Drugstore,Dog Run,Discount Store,Diner
